## Steps
1. We will read the data set directly here and we will clean and work from here
2. Step 1: We will load the modules needed to work
3. Step 2: We will keep the data set from a github repository where we have kept it
4. Step 3: We will clean the data set using tidy data principles so that our data set will have a limited set of variables.

## What is a tidy data set?
A tidy data set is a data set that has three properties:

- It has only information about the individuals in the rows
- It has only information about the variables in the columns
- It has only ONE information per cell
This concept is attributed to Hadley Wickham. We are using here Python to scrape and clean the data set.

In [0]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
#From pandas import Series, DataFrame
import json
import csv
import requests
import urllib3
import urllib
import io
import statsmodels as sm

In [0]:
url = "https://raw.githubusercontent.com/arinbasu/hlth460/master/survey.csv"
data = requests.get(url).content
mydata = pd.read_csv(io.StringIO(data.decode('utf-8')))

In [0]:
mydata.head() # shows all variables

In [0]:
mydata.columns # returns a list of the column names
# We will create a smaller data set by removing 'Timestamp', and 'Comments'
mydata1 = mydata.drop(['Timestamp', 'comments'], axis = 1) # dropped two variables
mydata1['Age'].count() # total number of observations
mydata1['age_rec'] = pd.cut(mydata['Age'], bins = [mydata1['Age'].min(), 10, 20, 30, 40, 50, mydata1['Age'].max()])
# mydata1['age_rec'].value_counts() # provides with a listing of the counts of diferent categories
# we see that age group -1726 to 10 is not possible if these are technology workers of working age, so we will remove these
pd.Categorical(mydata1['age_rec'])
mydata['age_rec1'] = mydata1['age_rec'].cat.rename_categories(['lt 10', '10-19', '20-29', '30-39', '40-49', 'gt 50']) # need to add the keyword 'cat'
mydata1['age_rec1'].value_counts()
mydata1['age_rec2'] = mydata1.age_rec1[mydata1.age_rec1 != 'lt 10']
mydata1['age_rec2'].value_counts()
mydata2 = mydata1.drop(['age_rec', 'age_rec1'], axis = 1) # remove the incoherent age groups
mydata2.columns # gives the variable names

# count the Gender
mydata2['Gender'].value_counts() # gives us the tally of gender categories
#pd.Categorical(mydata2['Gender']) # it is impossible to correct it easily here
# In these cases, use a spreadsheet to correct using search and find

In [0]:
# In case of recoding the 49 categories of Gender, we will adopt the following logic
# x is Gender
# if x == 'M' | 'Male', then y == 'Male',
# else if x == 'F' | 'Female', then y == 'Female',
# else y == 'Other'
# then we will do mydata2['gender'] = mydata2.Gender.apply(myfunc)

def myfunc(g):
  
  if (g == "M"):
    return 'Male'
  elif (g == 'Male'):
    return 'Male'
  elif(g == "F"):
    return 'Female'
  elif(g == "Female"):
    return 'Female'
  else:
    return "Others"
  

  

mydata2['gender'] = mydata2.Gender.apply(myfunc) 
mydata2['gender'].value_counts()

#mydata2['gender'].count()
  

In [0]:
# what columns now?
mydata2.columns

## Our idea
At this time, let's try to understand something from the data. Let's say we are interested to find out if mental health interferes with their work. Let's also say that we suspect or we'd like to test evidence from this cross-sectional survey of 1259 people whether their participation in a wellness programme is associated with their perception that their mental health interferes with their work. Albeit true that there will be reverse causation as we do not have any way of knowing from this dataset whether people who attended a wellness programme because of their work interefered by their mental health status or whether their mental health status was affected and they participated in a wellness programme because of this but this existence of the association will tell us a story. We will also like to find out if this varies with country, and the size of the company, and self employment status or whether this is more common among remote workers than those work at the office. 

## Strategy
Therefore we will drop several variables and work with only the following variables in the data set:

- age_rec2
- gender
- Country
- self_employed
- family_history
- treatment
- work_interfere
- no_employees
- remote_work



In [0]:
newdata = mydata2[['age_rec2', 'gender', 'Country', 'self_employed', 'family_history', 'treatment', 'work_interfere', 'no_employees', 'remote_work']] # create a subset small data set

In [0]:
newdata.describe()

In [0]:
newdata['work_interfere'].value_counts() # this gives us the tally of how often does mental health issues interefer with their work
newdata['work_interf'] = newdata['work_interfere'].astype('category') # set it to a categorical variable
pd.Categorical(newdata['work_interf']) # get a listing
newdata['work_interf'].dtype

In [0]:
def work(g):
  
  if (g == "Never"):
    return 'No'
  elif (g == 'Often'):
    return 'Yes'
  elif(g == "Rarely"):
    return 'No'
  else:
    return 'Yes'
  

newdata['interfere'] = newdata['work_interf'].apply(work)

newdata['interfere'].value_counts()

In [0]:
newdata.describe()

# Create a smaller data set

newdata_small = newdata[['age_rec2', 'gender', 'Country', 'self_employed',
                        'family_history', 'treatment',
                        'no_employees',
                        'remote_work',
                        'interfere']]

In [0]:
# Let's run some analysis
# Let's get the value counts
newdata_small['age_rec2'].value_counts()
newdata_small['gender'].value_counts()
countrycount = pd.DataFrame(newdata_small['Country'].value_counts()) # a data frame of list of countries with counts
countrycount

countrycount.Country[countrycount['Country'] > 5].index # then which countries have more than 5 respondents

mydata = newdata_small.loc[newdata_small['Country'].isin(countrycount.Country[countrycount['Country'] > 5].index)] # a data subset of all respondents from countries > 5 respondents

mydata.describe()
mydata2 = mydata[['age_rec2', 'gender', 'self_employed', 'family_history', 'treatment', 'no_employees', 'remote_work', 'interfere']] # take a smaller subset

In [0]:
self_empl = pd.DataFrame(mydata2['self_employed'].value_counts())
interf1 = pd.DataFrame(mydata2['interfere'].value_counts())


In [0]:
age1 = pd.crosstab(mydata2['age_rec2'], mydata2['interfere'],
           margins = True)
age1['yespct'] = age1['Yes'] * 100/age1['All']
age1

sex = pd.crosstab(mydata2['gender'], mydata2['interfere'],
           margins = True)
sex['yespct'] = sex['Yes'] * 100/sex['All']
sex

## Write a function to do crosstabs and present the tables

